<a href="https://colab.research.google.com/github/Dipanshu712/Machine_Learning_Proj/blob/main/Player_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install ultralytics opencv-python deep_sort_realtime --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 839.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.1 MB/s eta 0:00:00


In [21]:
# Fast Global ID Video Output Pipeline (no cropping)
# - Detect & track players in both videos
# - Assign consistent global IDs using histogram similarity
# - Render videos with same ID for matching players

import os
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from sklearn.metrics.pairwise import cosine_similarity

model = YOLO("/content/drive/MyDrive/Colab Notebooks/best.pt")
tracker = DeepSort(max_age=30)

videos = {
    "broadcast": "/content/drive/MyDrive/broadcast.mp4",
    "tacticam": "/content/drive/MyDrive/tacticam.mp4"
}

output_dir = "/content/video_output"
os.makedirs(output_dir, exist_ok=True)

# Store visual features per camera
features_db = {"broadcast": {}, "tacticam": {}}

# Step 1: Detect, track, and extract feature vectors for each ID
def extract_features_and_tracks(video_path, cam_name):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_path = os.path.join(output_dir, f"{cam_name}_global_ids.mp4")
    out_video = cv2.VideoWriter(out_path, fourcc, fps, (frame_width, frame_height))

    id_to_hist = {}
    frame_num = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        detections = []
        results = model(frame)
        for r in results:
            boxes = r.boxes.xyxy.cpu().numpy()
            confs = r.boxes.conf.cpu().numpy()
            classes = r.boxes.cls.cpu().numpy()
            for box, conf, cls in zip(boxes, confs, classes):
                if int(cls) in [0, 3]:
                    x1, y1, x2, y2 = map(int, box)
                    detections.append(((x1, y1, x2, y2), float(conf), "player"))

        tracks = tracker.update_tracks(detections, frame=frame)

        for track in tracks:
            if not track.is_confirmed():
                continue
            track_id = track.track_id
            x1, y1, x2, y2 = map(int, track.to_ltrb())
            h, w, _ = frame.shape
            x1, y1, x2, y2 = max(0, x1), max(0, y1), min(w, x2), min(h, y2)
            crop = frame[y1:y2, x1:x2]
            if crop.size > 0:
                crop_resized = cv2.resize(crop, (64, 128))
                hist = cv2.calcHist([crop_resized], [0,1,2], None, [8,8,8], [0,256]*3)
                hist = cv2.normalize(hist, hist).flatten()
                features_db[cam_name][track_id] = hist
                id_to_hist[track_id] = hist
                cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
                cv2.putText(frame, f"LocalID: {track_id}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

        out_video.write(frame)
        frame_num += 1

    cap.release()
    out_video.release()
    print(f"✅ Processed {cam_name}, saved video to: {out_path}")

# Extract per-camera features
for cam, path in videos.items():
    extract_features_and_tracks(path, cam)

# Step 2: Match tacticam IDs to broadcast global IDs
def match_ids(broadcast_feats, tacticam_feats):
    global_id_map = {}
    for tac_id, tac_feat in tacticam_feats.items():
        best_match = None
        best_score = -1
        for bro_id, bro_feat in broadcast_feats.items():
            score = cosine_similarity([tac_feat], [bro_feat])[0][0]
            if score > best_score:
                best_score = score
                best_match = bro_id
        global_id_map[tac_id] = best_match
    return global_id_map

id_mapping = match_ids(features_db["broadcast"], features_db["tacticam"])

# Save the ID mapping to CSV
map_df = pd.DataFrame(list(id_mapping.items()), columns=["tacticam_id", "global_id"])
map_df.to_csv("/content/global_id_mapping.csv", index=False)
print("\n✅ Global ID mapping saved: global_id_mapping.csv")



0: 384x640 3 players, 3354.9ms
Speed: 4.2ms preprocess, 3354.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 2932.0ms
Speed: 4.9ms preprocess, 2932.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 players, 2330.7ms
Speed: 3.8ms preprocess, 2330.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 2419.7ms
Speed: 4.9ms preprocess, 2419.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 players, 2360.6ms
Speed: 4.1ms preprocess, 2360.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 players, 1 referee, 3028.2ms
Speed: 3.2ms preprocess, 3028.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 goalkeeper, 12 players, 1 referee, 3141.7ms
Speed: 3.7ms preprocess, 3141.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 1 goalkeeper, 14 players,